<a href="https://colab.research.google.com/github/Byrnesz/point-of-interest-data/blob/master/Join_POI_with_Census_Block_Groups_(point_in_polygon_geospatial_joins_in_python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://global-uploads.webflow.com/5baafc2653bd67278f206724/5be267a03f7813daf821b31e_safegraph-logo-hidpi%403x-p-500.png)

# Join Point-Of-Interest Data To Census Data
### (How To Point-In-Polygon Geospatial Join in Python)

--------------
**[Ryan Fox Squire](https://www.linkedin.com/in/ryanfoxsquire/) | Product Data Scientist, [SafeGraph](https://safegraph.com/?utm_source=content&utm_medium=referral&utm_campaign=colabnotebook&utm_content=point-in-poly-join)**


Jan 2020

--------------
*Share this notebook: [Shareable Link](https://colab.research.google.com/drive/1Kt3vPVIQJUq4QeJ-rE08URpIJEr1g2H3#offline=true&sandboxMode=true)*

In [0]:
# special libraries to allow file access
from google.colab import drive as mountGoogleDrive 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


## **How to use this notebook**:

---

**Quick Start:** 
1. You can run this notebook using some data I've prepared.  You only need to do one thing: Run the next cell, and follow the prompt to authenticate your google account.
2. **Everything else will just run! Enjoy!**


*p.s. FYI -- GeoPandas does not come installed in GoogleColab by default, so some of the cells below will install some libraries in your Google Colab session.*




In [0]:
# These commands allow you to read directly from SafeGraph's public GoogleDrive containing Census Data and Sample Data
auth.authenticate_user()  # Authenticate and create the PyDrive client. 
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("You are fully authenticated and can edit and re-run everything in the notebook. Enjoy!")

You are fully authenticated and can edit and re-run everything in the notebook. Enjoy!


In [0]:
%%time 
# Install Geopandas (for spatial joins) onto Google CoLab
# HT: https://github.com/geopandas/geopandas/issues/901#issuecomment-458390318

# Install Gdal - rtree pre-req
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-

In [0]:
import pandas as pd
import geopandas as gpd

In [0]:
def pd_read_csv_drive(id, drive, dtype=None):
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('Filename.csv')  
  return(pd.read_csv('Filename.csv',dtype=dtype))

def get_drive_id(filename):
    # Note: OpenCensusData public GDrive folder: https://drive.google.com/open?id=1btSS6zo7_wJCCXAigkbhnaoeU-Voa9pG
    # Note: Sample of Geometry data public GDrive folder: https://drive.google.com/open?id=1PRYLkWNlO-_5EXwFmUZbYbtr_UJmKjBq
    drive_ids = {'cbg.geojson' : '19FIPhdpDTw_hKZemRvqu2V6OiFn5KjQ1',
                 'sample_safegraph_places' : '1zgd13vjMlhiPDw1tsiR93lW3whBa-yOA' #'1PRYLkWNlO-_5EXwFmUZbYbtr_UJmKjBq'
                 }
    return(drive_ids[filename])

### Load some SafeGraph Places Data


We will work with ~1000 POIs as an example. 

If you want to load in your own data, [you have several options. ](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92)

If the dataset is small, you can also upload it through the Google CoLab UI on the left. 

Or you can just use the demo data I have prepared. 

In [0]:
# If you want to use your own data, upload a csv through the Google CoLab UI on the left. 
# Then use some code like the following:

# my_uploaded_filename = 'MySafeGraphData.csv.gz'
# places_df = pd.read_csv(my_uploaded_filename)

In [0]:
# Use SafeGraph Demo Data
places_df = pd_read_csv_drive(get_drive_id('sample_safegraph_places'), drive=drive)
sg_places = places_df.iloc[0:1000].copy()
sg_places.head()

,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours,category_tags,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,iso_country_code,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,visitor_home_cbgs,visitor_work_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type
0,sg:bfb88a1910434b169e5733a7015eb192,sg:d65e327ffc174e558b1c4450da1b47f2,SG_BRAND_8cebb51fb05f1bf005e53bfec08d6c10,The Home Depot,The Home Depot,Building Material and Supplies Dealers,Hardware Stores,444130,33.988801,-117.704133,14549 Ramona Ave,Chino,CA,91710,"{ ""Mon"": [[""6:00"", ""22:00""]], ""Tue"": [[""6:00"",...",NaN,POLYGON ((-117.7045127749443 33.98971909914596...,OWNED_POLYGON,1.909394e+10,False,NaN,US,1.575158e+09,1.577837e+09,2415.0,1554.0,"[108,83,59,66,90,101,136,114,104,79,77,76,89,9...","{""060710019031"":77,""060710001162"":42,""06071000...","{""060710005043"":33,""060710001152"":32,""06071000...","{""US"":1529}",5272.0,19.0,"{""<5"":53,""5-20"":1277,""21-60"":805,""61-240"":214,...","{""McDonald's"":10}","{""Starbucks"":44,""Target"":38,""Costco Wholesale ...","[4,4,4,4,3,14,46,96,157,241,295,333,351,357,35...","{""Monday"":382,""Tuesday"":301,""Wednesday"":201,""T...","{""android"":957,""ios"":597}"
1,sg:c5f302af4c5b4dfe9c88d4ac20b22f09,sg:cfe9f6e3a75c44d583c59b14817f6c32,SG_BRAND_8cebb51fb05f1bf005e53bfec08d6c10,The Home Depot,The Home Depot,Building Material and Supplies Dealers,Hardware Stores,444130,33.912652,-83.444808,1740 Epps Bridge Pkwy,Athens,GA,30606,"{ ""Mon"": [[""6:00"", ""22:00""]], ""Tue"": [[""6:00"",...",NaN,POLYGON ((-83.44516396522522 33.91324411319755...,OWNED_POLYGON,1.706354e+10,False,NaN,US,1.575158e+09,1.577837e+09,2549.0,1738.0,"[133,73,80,86,60,78,90,105,75,74,65,71,62,122,...","{""132190301002"":91,""132190303001"":59,""13219030...","{""132190301001"":33,""132190301002"":25,""13219030...","{""US"":1712}",18250.0,20.0,"{""<5"":50,""5-20"":1341,""21-60"":830,""61-240"":208,...","{""Lowe's"":14,""Tires Plus"":8,""Academy Sports + ...","{""Walmart"":50,""Chick-fil-A"":47,""Kroger"":41,""Lo...","[11,11,15,13,35,72,88,131,215,285,341,450,443,...","{""Monday"":383,""Tuesday"":358,""Wednesday"":216,""T...","{""android"":860,""ios"":879}"
2,sg:f7468238732b45d9bf7f662efdc342a0,NaN,SG_BRAND_8cebb51fb05f1bf005e53bfec08d6c10,The Home Depot,The Home Depot,Building Material and Supplies Dealers,Hardware Stores,444130,30.350485,-91.028705,18139 Highland Rd,Baton Rouge,LA,70810,"{ ""Mon"": [[""6:00"", ""22:00""]], ""Tue"": [[""6:00"",...",NaN,POLYGON ((-91.02857887744904 30.35129397140938...,OWNED_POLYGON,1.225755e+10,False,NaN,US,1.575158e+09,1.577837e+09,1919.0,1265.0,"[84,59,54,58,51,54,64,111,62,49,59,57,71,86,77...","{""220050302031"":102,""220330045101"":63,""2203300...","{""220330040092"":40,""220050303003"":20,""22005030...","{""US"":1235}",8702.0,19.0,"{""<5"":44,""5-20"":1069,""21-60"":592,""61-240"":132,...","{""Walmart"":7}","{""Walmart"":56,""Shell Oil"":36,""Walgreens"":33,""R...","[8,7,5,4,5,27,55,105,166,220,267,330,342,338,3...","{""Monday"":312,""Tuesday"":278,""Wednesday"":180,""T...","{""android"":516,""ios"":750}"
3,sg:19d8ab8d4ed34c5e89b1ec29371e1057,NaN,SG_BRAND_8cebb51fb05f1bf005e53bfec08d6c10,The Home Depot,The Home Depot,Building Material and Supplies Dealers,Hardware Stores,444130,39.058327,-76.960460,2300 Broadbirch Dr,Silver Spring,MD,20904,"{ ""Mon"": [[""6:00"", ""22:00""]], ""Tue"": [[""6:00"",...",NaN,"POLYGON ((-76.960631 39.05891, -76.960308 39.0...",OWNED_POLYGON,1.301680e+10,False,NaN,US,1.575158e+09,1.577837e+09,1879.0,1161.0,"[61,61,62,60,67,60,87,65,60,64,65,60,59,72,81,...","{""240317014181"":25,""240317014141"":21,""24031701...","{""240317014211"":14,""240317015061"":7,""240

In [0]:
%%time
# SLOW -- this is a large GeoJSON, so loading is a slow step. ~ 4min 33seconds
downloaded = drive.CreateFile({'id':get_drive_id('cbg.geojson')}) 
downloaded.GetContentFile('CBG_GEOS.csv')  
cbg_geos = gpd.read_file('CBG_GEOS.csv')
print(cbg_geos.shape)
print(cbg_geos.head())

(220333, 9)
  StateFIPS  ...                                           geometry
0        01  ...  MULTIPOLYGON (((-85.37282 32.63424, -85.37275 ...
1        01  ...  MULTIPOLYGON (((-85.38346 32.64838, -85.38301 ...
2        01  ...  MULTIPOLYGON (((-85.37139 32.60139, -85.37138 ...
3        01  ...  MULTIPOLYGON (((-86.64797 33.59205, -86.64771 ...
4        01  ...  MULTIPOLYGON (((-86.65206 33.59869, -86.65204 ...

[5 rows x 9 columns]
CPU times: user 5min, sys: 14.7 s, total: 5min 14s
Wall time: 7min 33s


In [0]:
print(cbg_geos.shape)
cbg_geos.head()

(220333, 9)


,StateFIPS,CountyFIPS,TractCode,BlockGroup,CensusBlockGroup,State,County,ClassCode,geometry
0,01,081,041600,1,010810416001,AL,Lee County,H1,"MULTIPOLYGON (((-85.37282 32.63424, -85.37275 ..."
1,01,081,041600,2,010810416002,AL,Lee County,H1,"MULTIPOLYGON (((-85.38346 32.64838, -85.38301 ..."
2,01,081,041700,4,010810417004,AL,Lee County,H1,"MULTIPOLYGON (((-85.37139 32.60139, -85.37138 ..."
3,01,073,011107,4,010730111074,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.64797 33.59205, -86.64771 ..."
4,01,073,011108,4,010730111084,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.65206 33.59869, -86.65204 ..."


### Convert SafeGraph Data into a GeoPandas DataFrame

In [0]:
%%time
places_gpd = sg_places.copy()
print(places_gpd.shape)
places_gpd = places_gpd[places_gpd.iso_country_code=='US'][['safegraph_place_id', 'latitude', 'longitude']]
print(places_gpd.shape)
places_gpd = gpd.GeoDataFrame(places_gpd, geometry=gpd.points_from_xy(places_gpd.longitude, places_gpd.latitude))
places_gpd.crs = {'init' :'epsg:4326'} 
places_gpd = places_gpd[places_gpd.geometry.type == 'Point'][['safegraph_place_id', 'geometry']]
print(places_gpd.shape)

(1000, 38)
(1000, 3)
(1000, 2)
CPU times: user 52.2 ms, sys: 2.91 ms, total: 55.1 ms
Wall time: 66.2 ms


/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


### Rename the CBG Geometry data columns for clarity 

In [0]:
print(cbg_geos.shape)
cbgs = cbg_geos.rename(columns={'CensusBlockGroup':'census_block_group'})[['census_block_group', 'geometry']]
cbgs = cbgs[cbgs.geometry.type.isin(['Polygon', 'MultiPolygon'])]
print(cbgs.shape)
cbgs.head()

(220333, 9)
(220333, 2)


,census_block_group,geometry
0,010810416001,"MULTIPOLYGON (((-85.37282 32.63424, -85.37275 ..."
1,010810416002,"MULTIPOLYGON (((-85.38346 32.64838, -85.38301 ..."
2,010810417004,"MULTIPOLYGON (((-85.37139 32.60139, -85.37138 ..."
3,010730111074,"MULTIPOLYGON (((-86.64797 33.59205, -86.64771 ..."
4,010730111084,"MULTIPOLYGON (((-86.65206 33.59869, -86.65204 ..."


### Do GeoSpatial Join (Slow)

There are ways to optimize this if you are running at scale in production, but if you are doing ad-hoc analysis, this works with a little patience. 

In [0]:
%%time
# SLOW STEP. Spatial Join.
poi_cbgs = gpd.sjoin(places_gpd, cbgs, how="left", op='intersects')
print(poi_cbgs.shape)
print(poi_cbgs[~poi_cbgs.census_block_group.isna()].shape)
print(poi_cbgs.head())

/usr/local/lib/python3.6/dist-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


(1000, 4)
(998, 4)
                    safegraph_place_id  ... census_block_group
0  sg:bfb88a1910434b169e5733a7015eb192  ...       060710005043
1  sg:c5f302af4c5b4dfe9c88d4ac20b22f09  ...       132190302001
2  sg:f7468238732b45d9bf7f662efdc342a0  ...       220330040091
3  sg:19d8ab8d4ed34c5e89b1ec29371e1057  ...       240317014211
4  sg:5c8e92835222419b801b2969493b3134  ...       170318316001

[5 rows x 4 columns]
CPU times: user 22.1 s, sys: 92.8 ms, total: 22.2 s
Wall time: 22.3 s


### Save For Later! Your mapping between `safegraph_place_id` and `census_block_group` FIPS. 

In [0]:
from google.colab import files
write_file_path = "poi_sgpids_cbgs_map.csv"
sgpids_cbgs = poi_cbgs[['safegraph_place_id', 'census_block_group']]
sgpids_cbgs.to_csv(write_file_path, index=False)
files.download(write_file_path)